In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki

## Get index of articles

* [all indices on Wikipedia](https://en.wikipedia.org/wiki/Portal:Contents/Indices)
* topics not searched
* international trade ("topics"), theory of constraints (small)
* too big: mathematics, neuroscience

In [ ]:
path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [ ]:
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology',
           'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']

In [ ]:
links = {}
for topic in topics:
    dump.load_page('Index of %s articles' % topic)
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
# https://en.wikipedia.org/wiki/Lists_of_mathematics_topics
# algebra
math_topics = ['calculus', 'geometry', 'abstract algebra',
               'Boolean algebra', 'commutative algebra',# 'homological algebra',
               'group theory',# 'representation theory', 
               'linear algebra']
# calculus & analysis
# math_topics += ['complex analysis', 'functional analysis',
#                 'integration and measure theory', 'harmonic analysis',
#                 'Fourier analysis', 'multivariable calculus', 'real analysis',
#                 'variational']
# geometry
# math_topics += ['geometry', 'curves', 'triangle', 'circle', 'general topology',
#                 'differential geometry', 'algebraic geometry', 'algebraic topology',
#                 'geometric topology', 'know theory', 'Lie groups']
# number theory
math_topics += [#'algebraic number theory',
                'number theory']
# applied math
math_topics += ['dynamical systems and differential equations']
#                 'partial differential equation']
topics += math_topics

In [ ]:
for topic in math_topics:
    dump.load_page('List of %s topics' % topic)
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
# topics += ['physics']

In [ ]:
# import string

# topic = 'physics'
# links[topic] = []
# for letter in ['!$@', '0–9'] + list(string.ascii_uppercase):
#     dump.load_page('Index of physics articles (%s)' % letter)
#     links[topic].extend([str(l) for l in dump.article_links])
# print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
# topics += ['mathematics']

In [ ]:
# topic = 'mathematics'
# links[topic] = []
# for letter in ['0–9'] + list(string.ascii_uppercase):
#     dump.load_page('Wikipedia:WikiProject Mathematics/List of mathematics articles (%s)'
#                    % letter)
#     links[topic].extend([str(l) for l in dump.article_links])
# print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

## Build graphs of topics

### 1 network per topic

In [ ]:
import pickle
import gensim.utils as gu

path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [ ]:
path_to_save = '/Users/harangju/Developer/data/wiki/graphs/dated/'

networks = {}
# for topic, ls in links.items():
for topic in math_topics:
    ls = links[topic]
    print('\nTopic: ' + topic)
    networks[topic] = wiki.Net()
    networks[topic].build_graph(name=topic, dump=dump, nodes=ls, model=tfidf, dct=dct)
    networks[topic].save_graph(path_to_save + topic + '.pickle')
    networks[topic].save_barcodes(path_to_save + topic + '.barcode')

## Generate null networks

In [ ]:
path_to_save = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_to_save + topic + '.pickle')

### Randomized target, year, & both?

Just randomizing year -> you get the same structures, it's just a matter of when you get those structures.
If you randomize year & target, then you're randomizing the structure & how they come about without changing any basic network statistics.

In [ ]:
null_models = ['target', 'year']
num_nulls = 2
nulls = {}
for null_model in null_models:
    print('Null model: ' + null_model)
    path_to_save_null = '/Users/harangju/Developer/data/wiki/graphs/null-'+null_model+'/'
    nulls[null_model] = {}
    for topic, network in networks.items():
        print('Topic: ' + topic)
        nulls[null_model][topic] = []
        for i in range(num_nulls):
            print('Null: ' + str(i))
            null = network.randomize(null_model)
            null.graph.name = topic+'-null-'+str(i)
            null.save_graph(path_to_save_null + null.graph.name + '.pickle')
            null.save_barcodes(path_to_save_null + null.graph.name + '.barcode')
            nulls[null_model][topic].append(null)

## Post hoc

In [ ]:
path = '/Users/harangju/Developer/data/wiki/graphs/dated/'
path_null_target = '/Users/harangju/Developer/data/wiki/graphs/null-target/'
path_null_year = '/Users/harangju/Developer/data/wiki/graphs/null-year/'
networks = {}
num_nulls = 2
null_target = {}
null_year = {}
for topic in topics:
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path + topic + '.pickle')
    null_target[topic] = [None for i in range(num_nulls)]
    null_year[topic] = [None for i in range(num_nulls)]
    for i in range(num_nulls):
        null_target[topic][i] = wiki.Net()
        null_target[topic][i].load_graph(path_null_target + topic +
                                         '-null-' + str(i) + '.pickle')
        null_year[topic][i] = wiki.Net()
        null_year[topic][i].load_graph(path_null_year + topic +
                                       '-null-' + str(i) + '.pickle')

In [ ]:
all_nets = list(networks.values()) + \
           [n for ns in null_target.values() for n in ns] + \
           [n for ns in null_year.values() for n in ns]
len(all_nets)

In [ ]:
for network in all_nets:
    print(network.graph.name)
    wiki.Net.assign_core_periphery(network.graph)

In [ ]:
networks['evolutionary biology'].graph.graph

In [ ]:
for topic in topics:
    networks[topic].save_graph(path + topic + '.pickle')
    for i in range(num_nulls):
        null_target[topic][i].save_graph(path_null_target + topic +
                                         '-null-' + str(i) + '.pickle')
        null_year[topic][i].save_graph(path_null_year + topic +
                                       '-null-' + str(i) + '.pickle')

Gephi notes
* node size, fruchterman reingold = [10, 40], force atlas 2 = [4 16]
* text size = [1 1.4]
* preview font size = 5